In [1]:
import pymc3 as pm
import theano.tensor as tt
import theano.tensor.slinalg as sla
import theano.tensor.nlinalg as nla
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

/home/nilavro/irleak/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
x , y = np.loadtxt( 'test_data.txt', delimiter=',', skiprows = 1,   unpack=True)




In [3]:
N_states = 2
N_chain = len(y)

# Transition probability stochastic
theta = np.ones((N_states,N_states)) + 1.
alphaA = np.ones(N_states)*2.1
betaA = np.ones(N_states)*1.1
alphaS = np.ones(N_states)*2.1
betaS = np.ones(N_states)*1.1

In [7]:
class HMMStatesN(pm.Categorical):
    """
    Hidden Markov Model States
    Parameters
    ----------
    P : tensor
        transition probability
        shape = (N_states,N_states)
        
    PA : tensor
         equilibrium probabilities
         shape = (N_states)
    
    """

    def __init__(self, PA=None, P=None, N_states=2,
                 *args, **kwargs):
        super(pm.Categorical, self).__init__(*args, **kwargs)
        self.P = P
        self.PA = PA
        self.k = N_states
        self.mode = tt.cast(0,dtype='int64')

    def logp(self, x):
        P = self.P
        PA = self.PA
        
        # calculate equilibrium
        
        # now we need to create an array with probabilities
        # so that for x=A: PA=P1, PB=(1-P1)
        # and for x=B: PA=P2, PB=(1-P2)
        #P = tt.switch(x[:-1],P1,P2)
        
        PS = P[x[:-1]]
                
        x_i = x[1:]
        ou_like = pm.Categorical.dist(PS).logp(x_i)
        return pm.Categorical.dist(PA).logp(x[0]) + tt.sum(ou_like)

In [8]:
class HMMGaussianEmissionsN(pm.Continuous):
    """
    Hidden Markov Model Gaussian Emissions
    Parameters
    ----------
    A : tensor
        prior for Gaussian emission mu
        shape = (2,N_states)
        
    S : tensor
        prior for Gaussian emission width
        shape = (2,N_states)
    
    states : tensor
         equilibrium probabilities
         shape = (N_states)
    
    """

    def __init__(self, A=None, S=None, states=None,
                 *args, **kwargs):
        super(HMMGaussianEmissionsN, self).__init__(*args, **kwargs)
        self.A = A
        self.S = S
        self.states = states
        self.mean = 0.

    def logp(self, x):
        A = self.A
        S = self.S
        states = self.states
        
        AS = A[states]        
        SS = S[states]
        
        ou_like = pm.Normal.dist(mu=AS,sd=SS).logp(x)
        return tt.sum(ou_like)

In [28]:
len(y)

1000

In [31]:
from scipy import optimize
with pm.Model() as model:
    # N_states state model
    #P = tt.stack( [pm.Dirichlet('P_'+str(i), a=np.ones(N_states)) for i in rangdisaster_modele(N_states)] )
    
    P = pm.Dirichlet('P', a=np.ones((N_states,N_states)), shape=(N_states,N_states))
    A = pm.InverseGamma('A',alpha=alphaA, beta=betaA, shape=(N_states))
    S = pm.InverseGamma('S', alpha=alphaS, beta=betaS, shape=(N_states))
    
#    Pfull = tt.dmatrix('Pfull')
#    PA = tt.dmatrix('PA')
    AA = tt.dmatrix('AA')
        
    AA = tt.eye(N_states) - P + tt.ones(shape=(N_states,N_states))
    
    PA = pm.Deterministic('PA',sla.solve(AA.T,tt.ones(shape=(N_states))))
    
    states = HMMStatesN('states',PA=PA, P=P, N_states=N_states, shape=(len(y),))
    
    emission = HMMGaussianEmissionsN('emission',A = A, S = S, states=states,observed = y)
    start = pm.find_MAP(fmin=optimize.fmin_powell)
    step1 = pm.Metropolis(vars=[P,A,S,PA,emission])
    step2 = pm.CategoricalGibbsMetropolis(vars=[states])
    trace = pm.sample(10, start=start, step=[step1,step2])    
    

/home/nilavro/irleak/lib/python3.5/site-packages/pymc3/tuning/starting.py:92: UserWarning: In future versions, set the optimization algorithm with a string. For example, use `method="L-BFGS-B"` instead of `fmin=sp.optimize.fmin_l_bfgs_b"`.
  warnings.warn('In future versions, set the optimization algorithm with a string. '
logp = -1,408.9:   5%|▍         | 230/5000 [00:01<00:24, 193.07it/s]

Optimization terminated successfully.
         Current function value: 1408.918917
         Iterations: 3
         Function evaluations: 235


logp = -1,408.9:   5%|▍         | 235/5000 [00:07<02:29, 31.78it/s] 
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>CompoundStep
>>Metropolis: [P_stickbreaking__]
>>Metropolis: [S_log__]
>>Metropolis: [A_log__]
>CategoricalGibbsMetropolis: [states]
100%|██████████| 510/510 [09:13<00:00,  1.09s/it]
The number of effective samples is smaller than 10% for some parameters.


In [30]:
with model:
    %time approx = pm.fit(method = 'advi')

  0%|          | 0/10000 [00:00<?, ?it/s]


IndexError: index 2 is out of bounds for size 2
Apply node that caused the error: AdvancedSubtensor1(P, Subtensor{:int64:}.0)
Toposort index: 82
Inputs types: [TensorType(float64, matrix), TensorType(int64, vector)]
Inputs shapes: [(2, 2), (999,)]
Inputs strides: [(8, 16), (8,)]
Inputs values: [array([[0.2485256 , 0.7514744 ],
       [0.62193988, 0.37806012]]), 'not shown']
Outputs clients: [[Sum{axis=[1], acc_dtype=float64}(AdvancedSubtensor1.0), InplaceDimShuffle{1,0}(AdvancedSubtensor1.0), Elemwise{true_div,no_inplace}(AdvancedSubtensor1.0, InplaceDimShuffle{0,x}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/home/nilavro/irleak/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2850, in run_ast_nodes
    if self.run_code(code, result):
  File "/home/nilavro/irleak/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File 

#  ROUGH 

In [13]:
import numpy as np
import pymc3 as pm
import pdb

In [14]:
data = np.loadtxt('test_data.txt',
                 dtype=np.dtype([('state', np.uint8),
                                 ('emission', np.float)]),
                 delimiter=',',
                 skiprows=1)

In [ ]:
def unconditionalProbability(Ptrans):
"""Compute the unconditional probability for the states of a Markov chain."""

    m = Ptrans.shape[0]

    P = np.column_stack((Ptrans, 1. - Ptrans.sum(axis=1)))

    I = np.eye(m)
    U = np.ones((m, m))
    u = np.ones(m)

    return np.linalg.solve((I - P + U).T, u)

In [ ]:
# Two state model for simplicity.
N_states = 2
N_chain = len(data)

In [ ]:
# Transition probability stochastic
theta = np.ones(N_states) + 1.

In [15]:
def Ptrans_logp(value, theta):
    logp = 0.
    for i in range(value.shape[0]):
        logp = logp + pm.Dirichlet.dist(value[i], theta)
    return logp

In [16]:
def Ptrans_random(theta):
    return pm.rdirichlet.dist(theta, size=len(theta))

In [25]:
Ptrans = pm.DensityDist.dist('Ptrans',  Ptrans_logp,         
                        parents={'theta': theta},
                        random=Ptrans_random)


beta = pymc.DensityDist('Ptrans', lambda value: -1.5 * T.log(1 + value**2), testval=0)


TypeError: __init__() got an unexpected keyword argument 'parents'

In [17]:
#Hidden states stochastic
def states_logp(value, Ptrans=Ptrans):
    
    if sum(value>1):
        return -np.inf

    P = np.column_stack((Ptrans, 1. - Ptrans.sum(axis=1)))

    Pinit = unconditionalProbability(Ptrans)

    logp = pymc.categorical_like(value[0], Pinit)

    for i in range(1, len(value)):
        try:
            logp = logp + pm.categorical_like(value[i], P[value[i-1]])
        except:
            pdb.set_trace()

    return logp

NameError: name 'Ptrans' is not defined

In [ ]:
def states_random(Ptrans=Ptrans, N_chain=N_chain):
   P = np.column_stack((Ptrans, 1. - Ptrans.sum(axis=1)))

   Pinit = unconditionalProbability(Ptrans)

   states = np.empty(N_chain, dtype=np.uint8)

   states[0] = pymc.rcategorical(Pinit)

   for i in range(1, N_chain):
       states[i] = pymc.rcategorical(P[states[i-1]])

   return states

states = pymc.Stochastic(logp=states_logp,
                        doc='Hidden states',
                        name='states',
                        parents={'Ptrans': Ptrans},
                        random=states_random,
                        dtype=np.uint8)

# Gaussian emission parameters
mu = pymc.Normal('mu', 0., 1.e-6, value=np.random.randn(N_states))
sigma = pymc.Uniform('sigma', 0., 100.,
value=np.random.rand(N_states))

y = pymc.Normal('y', mu[states], 1./sigma[states]**2,
value=data['emission'], observed=True)